In [1]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3 as sql
import re
import numpy as np
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt


%matplotlib inline

In [2]:
pd.set_option('display.max_rows', 500)

In [3]:
suicide_all = pd.read_csv("suicide_all.csv")

In [4]:
suicide_all

,Month,Precinct,Date,Created,Address,Incident Type,Priority
0,July,C,07-01-2021,14:08:06,1207 MCGAVOCK ST,SUICIDAL PERSON,2
1,July,C,07-01-2021,16:39:59,639 LAFAYETTE ST,SUICIDAL PERSON,2
2,July,C,07-01-2021,23:23:27,639 LAFAYETTE ST,SUICIDAL PERSON,2
3,July,C,07-02-2021,06:23:06,1221 BROADWAY,SUICIDAL PERSON,2
4,July,C,07-02-2021,11:07:14,705 DREXEL ST,SUICIDAL PERSON,2
...,...,...,...,...,...,...,...
1720,March,W,03-23-2022,06:41:49,6700 CABOT DR,SUICIDAL PERSON,2M
1721,March,W,03-27-2022,07:54:28,329 SUSANNAH CT,SUICIDAL PERSON,3
1722,March,W,03-28-2022,19:23:55,517 RIVER ROUGE DR,SUICIDAL PERSON,3
1723,March,W,03-29-2022,14:01:59,8109 BOONE TRCE,SUICIDAL PERSON,2M


In [5]:
suicide_all_10AM_10PM = suicide_all[suicide_all['Created'].between('10:00:00', '21:59:59')]
suicide_all_10AM_10PM

,Month,Precinct,Date,Created,Address,Incident Type,Priority
0,July,C,07-01-2021,14:08:06,1207 MCGAVOCK ST,SUICIDAL PERSON,2
1,July,C,07-01-2021,16:39:59,639 LAFAYETTE ST,SUICIDAL PERSON,2
4,July,C,07-02-2021,11:07:14,705 DREXEL ST,SUICIDAL PERSON,2
6,July,C,07-04-2021,21:59:08,500 REP JOHN LEWIS WAY N,SUICIDAL PERSON,2
10,July,C,07-07-2021,13:45:19,526 8TH AVE S,SUICIDAL PERSON,2
...,...,...,...,...,...,...,...
1718,March,W,03-21-2022,16:23:38,610 SYLVAN HEIGHTS WAY,SUICIDAL PERSON,3
1719,March,W,03-22-2022,11:33:02,2602 HILLMEADE DR,SUICIDAL PERSON,2M
1722,March,W,03-28-2022,19:23:55,517 RIVER ROUGE DR,SUICIDAL PERSON,3
1723,March,W,03-29-2022,14:01:59,8109 BOONE TRCE,SUICIDAL PERSON,2M


In [6]:
suicide_all_10PM_10AM = suicide_all[~suicide_all['Created'].between('10:00:00', '21:59:59')]
suicide_all_10PM_10AM

,Month,Precinct,Date,Created,Address,Incident Type,Priority
2,July,C,07-01-2021,23:23:27,639 LAFAYETTE ST,SUICIDAL PERSON,2
3,July,C,07-02-2021,06:23:06,1221 BROADWAY,SUICIDAL PERSON,2
5,July,C,07-03-2021,03:37:59,639 LAFAYETTE ST,SUICIDAL PERSON,2
7,July,C,07-05-2021,01:03:16,MCGAVOCK ST / 10TH AVE S,SUICIDAL PERSON,2M
8,July,C,07-06-2021,22:41:17,JEFFERSON ST / 4TH AVE N,SUICIDAL PERSON,2
...,...,...,...,...,...,...,...
1714,March,W,03-10-2022,01:19:55,4419 TENNESSEE AVE,SUICIDAL PERSON,2M
1716,March,W,03-15-2022,09:56:04,4500 POST RD,SUICIDAL PERSON,3
1717,March,W,03-19-2022,22:05:50,5731 RIVER RD,SUICIDAL PERSON,2M
1720,March,W,03-23-2022,06:41:49,6700 CABOT DR,SUICIDAL PERSON,2M


# Central - Night

In [7]:
S_Calls_C_Night = suicide_all_10PM_10AM.loc[suicide_all_10PM_10AM["Precinct"]=="C"]

In [8]:
S_Calls_C_Night

,Month,Precinct,Date,Created,Address,Incident Type,Priority
2,July,C,07-01-2021,23:23:27,639 LAFAYETTE ST,SUICIDAL PERSON,2
3,July,C,07-02-2021,06:23:06,1221 BROADWAY,SUICIDAL PERSON,2
5,July,C,07-03-2021,03:37:59,639 LAFAYETTE ST,SUICIDAL PERSON,2
7,July,C,07-05-2021,01:03:16,MCGAVOCK ST / 10TH AVE S,SUICIDAL PERSON,2M
8,July,C,07-06-2021,22:41:17,JEFFERSON ST / 4TH AVE N,SUICIDAL PERSON,2
9,July,C,07-07-2021,05:20:48,8TH AVE S / CLARK PL,SUICIDAL PERSON,2
11,July,C,07-09-2021,23:39:10,623 6TH AVE S,SUICIDAL PERSON,2
14,July,C,07-10-2021,22:46:41,709 REP JOHN LEWIS WAY S,SUICIDAL PERSON,2
18,July,C,07-12-2021,02:22:34,4TH AVE S / BROADWAY,SUICIDAL PERSON,3
22,July,C,07-15-2021,03:47:35,709 REP JOHN LEWIS WAY S,SUICIDAL PERSON,2


# Hermitage - Night

In [9]:
S_Calls_H_Night = suicide_all_10PM_10AM.loc[suicide_all_10PM_10AM["Precinct"]=="C"]

In [10]:
S_Calls_H_Night

,Month,Precinct,Date,Created,Address,Incident Type,Priority
2,July,C,07-01-2021,23:23:27,639 LAFAYETTE ST,SUICIDAL PERSON,2
3,July,C,07-02-2021,06:23:06,1221 BROADWAY,SUICIDAL PERSON,2
5,July,C,07-03-2021,03:37:59,639 LAFAYETTE ST,SUICIDAL PERSON,2
7,July,C,07-05-2021,01:03:16,MCGAVOCK ST / 10TH AVE S,SUICIDAL PERSON,2M
8,July,C,07-06-2021,22:41:17,JEFFERSON ST / 4TH AVE N,SUICIDAL PERSON,2
9,July,C,07-07-2021,05:20:48,8TH AVE S / CLARK PL,SUICIDAL PERSON,2
11,July,C,07-09-2021,23:39:10,623 6TH AVE S,SUICIDAL PERSON,2
14,July,C,07-10-2021,22:46:41,709 REP JOHN LEWIS WAY S,SUICIDAL PERSON,2
18,July,C,07-12-2021,02:22:34,4TH AVE S / BROADWAY,SUICIDAL PERSON,3
22,July,C,07-15-2021,03:47:35,709 REP JOHN LEWIS WAY S,SUICIDAL PERSON,2


# South - Night

In [11]:
S_Calls_S_Night = suicide_all_10PM_10AM.loc[suicide_all_10PM_10AM["Precinct"]=="S"]

In [12]:
S_Calls_S_Night

,Month,Precinct,Date,Created,Address,Incident Type,Priority
164,July,S,07-03-2021,01:28:06,937 CURREY RD,SUICIDAL PERSON,2
167,July,S,07-08-2021,04:17:31,821 MURFREESBORO PIKE,SUICIDAL PERSON,2M
172,July,S,07-12-2021,04:36:00,160 ANTIOCH PIKE,SUICIDAL PERSON,2
174,July,S,07-15-2021,07:20:37,825 MURFREESBORO PIKE,SUICIDAL PERSON,2M
175,July,S,07-16-2021,02:06:15,133 HICKORY HOLLOW PL,SUICIDAL PERSON,2
177,July,S,07-20-2021,08:13:39,180 WALLACE RD,SUICIDAL PERSON,2
178,July,S,07-20-2021,08:54:36,180 WALLACE RD,SUICIDAL PERSON,2M
180,July,S,07-24-2021,09:24:04,213 ELBERTA ST,SUICIDAL PERSON,2M
182,July,S,07-28-2021,05:13:56,5170 HICKORY HOLLOW PKWY,SUICIDAL PERSON,2
184,July,S,07-30-2021,09:10:33,5115 HARDING PL,SUICIDAL PERSON,2


# North - Night

In [13]:
S_Calls_N_Night = suicide_all_10PM_10AM.loc[suicide_all_10PM_10AM["Precinct"]=="S"]

In [14]:
S_Calls_N_Night

,Month,Precinct,Date,Created,Address,Incident Type,Priority
164,July,S,07-03-2021,01:28:06,937 CURREY RD,SUICIDAL PERSON,2
167,July,S,07-08-2021,04:17:31,821 MURFREESBORO PIKE,SUICIDAL PERSON,2M
172,July,S,07-12-2021,04:36:00,160 ANTIOCH PIKE,SUICIDAL PERSON,2
174,July,S,07-15-2021,07:20:37,825 MURFREESBORO PIKE,SUICIDAL PERSON,2M
175,July,S,07-16-2021,02:06:15,133 HICKORY HOLLOW PL,SUICIDAL PERSON,2
177,July,S,07-20-2021,08:13:39,180 WALLACE RD,SUICIDAL PERSON,2
178,July,S,07-20-2021,08:54:36,180 WALLACE RD,SUICIDAL PERSON,2M
180,July,S,07-24-2021,09:24:04,213 ELBERTA ST,SUICIDAL PERSON,2M
182,July,S,07-28-2021,05:13:56,5170 HICKORY HOLLOW PKWY,SUICIDAL PERSON,2
184,July,S,07-30-2021,09:10:33,5115 HARDING PL,SUICIDAL PERSON,2


# Madison - Night

In [15]:
S_Calls_M_Night = suicide_all_10PM_10AM.loc[suicide_all_10PM_10AM["Precinct"]=="M"]

In [16]:
S_Calls_M_Night

,Month,Precinct,Date,Created,Address,Incident Type,Priority
107,July,M,07-17-2021,02:39:00,208 SLAYTON DR,SUICIDAL PERSON,2M
108,July,M,07-23-2021,05:26:19,705 TAHLENA AVE,SUICIDAL PERSON,2
109,July,M,07-24-2021,05:45:01,500 HOSPITAL DR,SUICIDAL PERSON,2
113,July,M,07-29-2021,09:43:39,510 HERITAGE DR,SUICIDAL PERSON,2M
298,August,M,08-06-2021,02:18:22,500 HOSPITAL DR,SUICIDAL PERSON,2
307,August,M,08-15-2021,23:04:27,313 NIX DR,SUICIDAL PERSON,3
309,August,M,08-17-2021,04:01:24,207 BEVERLY DR,SUICIDAL PERSON,2
312,August,M,08-25-2021,00:58:18,2232 GALLATIN PIKE N,SUICIDAL PERSON,2
313,August,M,08-31-2021,01:01:04,304 BECKLEA DR,SUICIDAL PERSON,2
495,September,M,09-22-2021,04:27:00,3303 HAWKWOOD LN,SUICIDAL PERSON,2


# Midtown Hills - Night

In [17]:
S_Calls_MT_Night = suicide_all_10PM_10AM.loc[suicide_all_10PM_10AM["Precinct"]=="MT"]

In [18]:
S_Calls_MT_Night

,Month,Precinct,Date,Created,Address,Incident Type,Priority
116,July,MT,07-05-2021,01:45:04,1510 DEMONBREUN ST,SUICIDAL PERSON,2
117,July,MT,07-06-2021,06:41:09,4199 NOLENSVILLE PIKE,SUICIDAL PERSON,2M
127,July,MT,07-17-2021,09:37:27,20TH AVE N / STATE ST,SUICIDAL PERSON,2
128,July,MT,07-19-2021,03:08:34,1121 HOLT HILLS PL,SUICIDAL PERSON,3
129,July,MT,07-19-2021,08:57:23,107 ACKLEN PARK DR,SUICIDAL PERSON,2
130,July,MT,07-19-2021,09:59:56,2316 WEST END AVE,SUICIDAL PERSON,2
132,July,MT,07-21-2021,07:37:52,107 ACKLEN PARK DR,SUICIDAL PERSON,2M
314,August,MT,08-02-2021,23:18:54,2000 CHURCH ST,SUICIDAL PERSON,2
316,August,MT,08-05-2021,01:56:09,21ST AVE S / CHILDRENS WAY,SUICIDAL PERSON,2M
319,August,MT,08-10-2021,23:53:50,2000 CHURCH ST,SUICIDAL PERSON,2M


# West - Night

In [19]:
S_Calls_W_Night = suicide_all_10PM_10AM.loc[suicide_all_10PM_10AM["Precinct"]=="W"]

In [20]:
S_Calls_W_Night

,Month,Precinct,Date,Created,Address,Incident Type,Priority
187,July,W,07-08-2021,22:19:58,645 OLD HICKORY BLVD,SUICIDAL PERSON,2
192,July,W,07-17-2021,22:27:56,4220 HARDING PIKE,SUICIDAL PERSON,3
197,July,W,07-25-2021,23:14:33,6624 CHARLOTTE PIKE,SUICIDAL PERSON,3
386,August,W,08-06-2021,22:01:35,6840 SONYA DR,SUICIDAL PERSON,2M
387,August,W,08-06-2021,23:49:07,213 SILO CT,SUICIDAL PERSON,2
390,August,W,08-22-2021,02:06:39,865 BELLEVUE RD,SUICIDAL PERSON,2
395,August,W,08-26-2021,05:03:11,255 OLD HICKORY BLVD,SUICIDAL PERSON,3
397,August,W,08-27-2021,09:00:52,7800 STEEPLECHASE LN,SUICIDAL PERSON,3M
605,September,W,09-25-2021,01:32:46,810 BELLEVUE RD,SUICIDAL PERSON,2
814,October,W,10-08-2021,09:45:52,4215 HARDING PIKE,SUICIDAL PERSON,2


# East - Night

In [21]:
S_Calls_E_Night = suicide_all_10PM_10AM.loc[suicide_all_10PM_10AM["Precinct"]=="E"]

In [22]:
S_Calls_E_Night

,Month,Precinct,Date,Created,Address,Incident Type,Priority
54,July,E,07-01-2021,08:59:04,525 LISCHEY AVE,SUICIDAL PERSON,3
61,July,E,07-15-2021,23:22:35,800 SYLVAN ST,SUICIDAL PERSON,3
257,August,E,08-20-2021,22:10:01,1007 SOLLEY DR,SUICIDAL PERSON,3
258,August,E,08-22-2021,09:25:31,4922 LOG CABIN RD,SUICIDAL PERSON,2M
259,August,E,08-23-2021,22:50:56,1107 SEYMOUR AVE,SUICIDAL PERSON,2M
262,August,E,08-27-2021,23:52:07,715 WOODLAND ST,SUICIDAL PERSON,2M
444,September,E,09-15-2021,22:28:56,3137 LARKSPUR DR,SUICIDAL PERSON,2
447,September,E,09-20-2021,09:59:34,1415 DICKERSON PIKE,SUICIDAL PERSON,2
652,October,E,10-03-2021,22:58:59,705 GLENVIEW DR,SUICIDAL PERSON,2
656,October,E,10-09-2021,22:14:09,2412 N 16TH ST,SUICIDAL PERSON,2


# Central - Day

In [23]:
S_Calls_C_Day = suicide_all_10AM_10PM.loc[suicide_all_10AM_10PM["Precinct"]=="C"]

In [24]:
S_Calls_C_Night

,Month,Precinct,Date,Created,Address,Incident Type,Priority
2,July,C,07-01-2021,23:23:27,639 LAFAYETTE ST,SUICIDAL PERSON,2
3,July,C,07-02-2021,06:23:06,1221 BROADWAY,SUICIDAL PERSON,2
5,July,C,07-03-2021,03:37:59,639 LAFAYETTE ST,SUICIDAL PERSON,2
7,July,C,07-05-2021,01:03:16,MCGAVOCK ST / 10TH AVE S,SUICIDAL PERSON,2M
8,July,C,07-06-2021,22:41:17,JEFFERSON ST / 4TH AVE N,SUICIDAL PERSON,2
9,July,C,07-07-2021,05:20:48,8TH AVE S / CLARK PL,SUICIDAL PERSON,2
11,July,C,07-09-2021,23:39:10,623 6TH AVE S,SUICIDAL PERSON,2
14,July,C,07-10-2021,22:46:41,709 REP JOHN LEWIS WAY S,SUICIDAL PERSON,2
18,July,C,07-12-2021,02:22:34,4TH AVE S / BROADWAY,SUICIDAL PERSON,3
22,July,C,07-15-2021,03:47:35,709 REP JOHN LEWIS WAY S,SUICIDAL PERSON,2


# Hermitage - Day

In [25]:
S_Calls_H_Day = suicide_all_10AM_10PM.loc[suicide_all_10AM_10PM["Precinct"]=="H"]

In [26]:
S_Calls_H_Day

,Month,Precinct,Date,Created,Address,Incident Type,Priority
67,July,H,07-01-2021,16:15:41,98 MAURY ST,SUICIDAL PERSON,2
68,July,H,07-01-2021,19:17:51,2634 MURFREESBORO PIKE,SUICIDAL PERSON,2
69,July,H,07-02-2021,13:03:19,1021 SUGAR CREEK CIR,SUICIDAL PERSON,2
70,July,H,07-02-2021,13:32:46,407 PITTS AVE,SUICIDAL PERSON,2M
72,July,H,07-03-2021,15:53:22,1524 LINCOYA BAY DR,SUICIDAL PERSON,2
73,July,H,07-04-2021,16:07:35,710 MURFREESBORO PIKE,SUICIDAL PERSON,2
75,July,H,07-06-2021,19:08:03,1414 TIMBER VALLEY DR,SUICIDAL PERSON,2M
76,July,H,07-06-2021,19:32:03,2500 MURFREESBORO PIKE,SUICIDAL PERSON,2
78,July,H,07-07-2021,21:56:19,87 MAURY ST,SUICIDAL PERSON,2M
80,July,H,07-08-2021,21:06:12,73 MAURY ST,SUICIDAL PERSON,2


# South - Day

In [27]:
S_Calls_S_Day = suicide_all_10AM_10PM.loc[suicide_all_10AM_10PM["Precinct"]=="S"]

In [28]:
S_Calls_S_Day

,Month,Precinct,Date,Created,Address,Incident Type,Priority
163,July,S,07-01-2021,10:51:58,2105 HICKORY HIGHLANDS DR,SUICIDAL PERSON,2
165,July,S,07-05-2021,18:28:01,660 BELL RD,SUICIDAL PERSON,2
166,July,S,07-06-2021,19:29:03,3001 HAMILTON CHURCH RD,SUICIDAL PERSON,2M
168,July,S,07-08-2021,14:42:27,2601 MURFREESBORO PIKE,SUICIDAL PERSON,2M
169,July,S,07-09-2021,11:20:13,2900 HAMILTON CHURCH RD,SUICIDAL PERSON,2M
170,July,S,07-09-2021,11:24:33,243 TUSCULUM RD,SUICIDAL PERSON,2
171,July,S,07-11-2021,15:45:03,2381 GRANNY WRIGHT LN,SUICIDAL PERSON,2M
173,July,S,07-12-2021,13:15:52,3960 LOGISTICS WAY,SUICIDAL PERSON,3
176,July,S,07-18-2021,12:19:40,521 HICKORY CLUB DR,SUICIDAL PERSON,3
179,July,S,07-22-2021,16:29:05,MURFREESBORO PIKE / FESSLERS LN,SUICIDAL PERSON,2


# North - Day

In [29]:
S_Calls_N_Day = suicide_all_10AM_10PM.loc[suicide_all_10AM_10PM["Precinct"]=="N"]

In [30]:
S_Calls_N_Day

,Month,Precinct,Date,Created,Address,Incident Type,Priority
138,July,N,07-01-2021,15:10:21,3428 W HAMILTON AVE,SUICIDAL PERSON,2
139,July,N,07-04-2021,19:46:19,1716 ROSA L PARKS BLVD,SUICIDAL PERSON,2
141,July,N,07-05-2021,16:26:09,1615 HEIMAN ST,SUICIDAL PERSON,3
143,July,N,07-07-2021,19:05:46,1027 ROSA L PARKS BLVD,SUICIDAL PERSON,2M
145,July,N,07-08-2021,14:39:09,2011 GOOCH ST,SUICIDAL PERSON,3
146,July,N,07-08-2021,16:49:44,208 HAYNES PARK DR,SUICIDAL PERSON,2
147,July,N,07-08-2021,17:41:15,303 W TRINITY LN,SUICIDAL PERSON,3
148,July,N,07-09-2021,18:48:48,2200 CHARLOTTE AVE,SUICIDAL PERSON,2
151,July,N,07-14-2021,12:24:16,515 SPRUCE ST,SUICIDAL PERSON,2
152,July,N,07-17-2021,19:41:40,2527 26TH AVE N,SUICIDAL PERSON,2


# Madison - Day

In [31]:
S_Calls_M_Day = suicide_all_10AM_10PM.loc[suicide_all_10AM_10PM["Precinct"]=="M"]

In [32]:
S_Calls_M_Day

,Month,Precinct,Date,Created,Address,Incident Type,Priority
101,July,M,07-01-2021,14:30:04,200 GALLATIN PIKE S,SUICIDAL PERSON,2
102,July,M,07-07-2021,10:37:28,501 STAR BLVD,SUICIDAL PERSON,2
103,July,M,07-09-2021,15:39:04,6102 MEADOWCREEK CIR,SUICIDAL PERSON,3
104,July,M,07-10-2021,19:38:05,1606 HUNTERS LN,SUICIDAL PERSON,2M
105,July,M,07-11-2021,21:06:02,1767 GALLATIN PIKE N,SUICIDAL PERSON,2
106,July,M,07-12-2021,13:59:18,801 GALLATIN PIKE S,SUICIDAL PERSON,2
110,July,M,07-25-2021,11:50:42,3458 DICKERSON PIKE,SUICIDAL PERSON,2
111,July,M,07-25-2021,21:58:30,2232 GALLATIN PIKE N,SUICIDAL PERSON,3
112,July,M,07-28-2021,20:42:58,2050 GALLATIN PIKE N,SUICIDAL PERSON,2
114,July,M,07-29-2021,11:15:17,510 HERITAGE DR,SUICIDAL PERSON,2M


# Midtown Hills - Day

In [33]:
S_Calls_MT_Day = suicide_all_10AM_10PM.loc[suicide_all_10AM_10PM["Precinct"]=="MT"]

In [34]:
S_Calls_MT_Day

,Month,Precinct,Date,Created,Address,Incident Type,Priority
118,July,MT,07-07-2021,13:24:33,2500 WEST END AVE,SUICIDAL PERSON,3
119,July,MT,07-07-2021,13:53:18,220 25TH AVE N,SUICIDAL PERSON,2
120,July,MT,07-07-2021,14:06:40,174 KNOLLS PL,SUICIDAL PERSON,2M
121,July,MT,07-07-2021,14:11:54,220 25TH AVE N,SUICIDAL PERSON,2
122,July,MT,07-12-2021,15:16:34,2403 NOLENSVILLE PIKE,SUICIDAL PERSON,2
123,July,MT,07-13-2021,12:11:38,782 OLD HICKORY BLVD,SUICIDAL PERSON,2
124,July,MT,07-13-2021,13:33:14,800 HONEYSUCKLE RD,SUICIDAL PERSON,2
125,July,MT,07-13-2021,18:08:53,1501 DEMONBREUN ST,SUICIDAL PERSON,2
126,July,MT,07-15-2021,15:00:35,1710 MAGNOLIA BLVD,SUICIDAL PERSON,2M
131,July,MT,07-19-2021,14:23:43,301 28TH AVE N,SUICIDAL PERSON,2


# West - Day

In [35]:
S_Calls_W_Day = suicide_all_10AM_10PM.loc[suicide_all_10AM_10PM["Precinct"]=="W"]

In [36]:
S_Calls_W_Day

,Month,Precinct,Date,Created,Address,Incident Type,Priority
185,July,W,07-03-2021,19:50:19,6500 CHARLOTTE PIKE,SUICIDAL PERSON,2
186,July,W,07-06-2021,21:45:23,7475 COCKRILL BEND BLVD,SUICIDAL PERSON,2M
188,July,W,07-09-2021,12:53:22,2215 ABBOTT MARTIN RD,SUICIDAL PERSON,3
189,July,W,07-09-2021,18:51:38,742 HARPETH PKWY W,SUICIDAL PERSON,3
190,July,W,07-10-2021,13:15:31,8634 OLD CHARLOTTE PIKE,SUICIDAL PERSON,3
191,July,W,07-15-2021,10:15:59,3900 CHARLOTTE AVE,SUICIDAL PERSON,2
193,July,W,07-18-2021,12:42:57,21 VAUGHNS GAP RD,SUICIDAL PERSON,2
194,July,W,07-22-2021,16:59:58,8428 BEAUTIFUL VALLEY DR,SUICIDAL PERSON,2
195,July,W,07-24-2021,19:42:24,4601 ALABAMA AVE,SUICIDAL PERSON,2
196,July,W,07-25-2021,13:30:37,4601 CHARLOTTE AVE,SUICIDAL PERSON,2M


# East - Day

In [37]:
S_Calls_E_Day = suicide_all_10AM_10PM.loc[suicide_all_10AM_10PM["Precinct"]=="E"]

In [38]:
S_Calls_E_Day

,Month,Precinct,Date,Created,Address,Incident Type,Priority
55,July,E,07-03-2021,20:33:39,2416 N 16TH ST,SUICIDAL PERSON,2M
56,July,E,07-06-2021,16:48:32,1118 INGLEWOOD DR,SUICIDAL PERSON,2M
57,July,E,07-07-2021,11:06:59,951 W EASTLAND AVE,SUICIDAL PERSON,2
58,July,E,07-07-2021,15:03:43,103 EASTMORELAND ST,SUICIDAL PERSON,2M
59,July,E,07-13-2021,12:56:04,820 RAMSEY ST,SUICIDAL PERSON,2M
60,July,E,07-14-2021,20:45:09,728 GALLATIN AVE,SUICIDAL PERSON,2M
62,July,E,07-16-2021,11:54:12,1600 HOLLY ST,SUICIDAL PERSON,3
63,July,E,07-28-2021,10:59:47,501 SUMMER PL,SUICIDAL PERSON,2M
64,July,E,07-28-2021,11:54:32,501 SUMMER PL,SUICIDAL PERSON,2M
65,July,E,07-28-2021,12:54:27,501 SUMMER PL,SUICIDAL PERSON,3
